# 7.1 전체구조
- Conv, Polling
- Affine - ReLU -> Conv - ReLU - (Pooling)

# 7.2 합성곱 계층
- Padding, Stride

## 7.2.1 완전연결 계층의 문제점
- Affine 
  - 데이터 형상이 무시된다. 
  - 이미지는 3차원 형상, 이 형상에는 소장한 공간적 정보가 담겨 있다.
- Conv
  - 형상을 유지
  - CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 수 있다.
  - 입출력 데이터 , feature map
    - input feature map
    - output feature map

## 7.2.2 합성곱 연산
- 필터 연산
  - 필터 == 커널
  - 합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용
    - 단일 곱셈 - 누산 (FMA, fused multiply-add)
  - 필터의 매개변수가 가중치에 해당됨
  - FMA 후에 편향을 더해줌   

## 7.2.3 패딩
- 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값으로 채움
- 출력 크기를 조절하기 위해서
- 합성곱 연산을 거칠 떄마다 크기가 작아지면 어느 시점에서는 출력 크기가 1이 되어버리는 문제

## 7.2.4 스트라이드
- 필터를 적용하는 위치의 간격 
  - 필터가 움직이는 간격
- 패딩, 스트라이드 -> 출력 크기
  - 입력 크기 (H, W)
  - 필터 크기 (FH, FW)
  - 출력 크기 (OH, OW)
  - 패딩 P, 스트라이드 S
  - $OH = \frac{H + 2P - FH}{S} + 1$
  - $OW = \frac{W + 2P - FW}{S} + 1$

## 7.2.5 3차원 데이터의 합성곱 연산
- 채널, 세로, 가로
- 입력 데이터의 채널 수와 필터의 채널 수가 같아야 함

## 7.2.6 블록으로 생각하기
- 3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉽다

- (C, H, W) $\otimes$ (C, FH, FW) -> (OH, OW)
  - 입력 데이터, 필터, 출력데이터 (한 장의 feature map)
- 합성곱 연산의 출력으로 다수의 채널을 내보내려면 필터를 다수 사용해야 함

- (C, H, W)  $\otimes$ (FN, C, FH, FW) -> (FN, OH, OW) 
  - 입력 데이터,  필터 , 출력 데이터( FN개의 feature map )

- 편향도 고려하면 다음과 같다.
- (C, H, W)  $\otimes$ (FN, C, FH, FW) -> (FN, OH, OW) + (FN, 1, 1) -> (FN, OH, OW)
  - 입력 데이터,  필터 , 출력 데이터( FN개의 feature map ), 편향, 출력 데이터

## 7.2.7 배치 처리
- 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장
- (N, C, H, W)  $\otimes$ (FN, C, FH, FW) -> (N, FN, OH, OW) + (FN, 1, 1) -> (N, FN, OH, OW)
- 주의할 점은, N회분의 처리가 한 번에 이루어진다는 것

# 7.3 풀링 계층
- 풀링은 새로, 가로 방향의 공간을 줄이는 연산
- max pooling
- 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 보통이다.
- average pooling
- 이미지 인식 분야에서는 주로 최대 풀링을 사용

## 7.3.1 풀링 계층의 특징
- 학습해야할 매개변수가 없다.
- 채널 수가 변하지 않는다.
- 입력의 변화에 영향을 적게 받는다 ( 강건하다 )

# 7.4 합성곱/풀링 구현하기

## 7.4.1 4차원 배열

In [2]:
import numpy as np

x = np.random.randn(10, 1, 28, 28)
# N, C, H, W x FN, C, H, W -> FN, OH, OW + FN, 1, 1 -> FN, OH, OW
x.shape

(10, 1, 28, 28)

## 7.4.2 im2col로 데이터 전개하기
- 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀜 
- im2col은 필터링하기 좋게 입력 데이터를 전개
- 입력 데이터에서 필터를 적용하는 영역(3차원 블록)을 한줄로 늘어놓는다.
- 필터의 적용 영역이 겹치는 경우가 대부분
- im2col로 전개한 후의 원소 수가 원래 블록의 원소 수보다 많아진다.
- 메모리를 더 많이 소비하는 단점
- 출력 결과는 2차원 행렬. 다시 4차원으로 변형해줘야 함

## 7.4.3 합성곱 계층 구현하기

In [3]:
import numpy as np
import sys, os
sys.path.append('../modules/')
from common.utils import im2col

x1 = np.random.rand(1, 3, 7, 7) # N, C, H, W
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # 필터가 적용되는 숫자 , C x H x W (필터의 원소 수)

(9, 75)


In [4]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # 필터가 적용되는 숫자 x N , C x H x W (필터의 원소 수)

(90, 75)


In [ ]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
    
  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2*self.pad- FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
    
    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T
    out = np.dot(col, col_W) + self.b
    
    # N , H, W, FN -> N, FN, H, W
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

## 7.4.4 풀링 계층 구현하기
- 합성곱 계층과 마찬가지로 im2col을 사용해 입력 데이터를 전개
- 채널쪽이 독립적이라는 점이 차이점

In [5]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad
    
  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)
    
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)
    
    out = np.max(col, axis=1)
    
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    
    return out

# 7.5 CNN 구현하기
Cov-ReLU-Pooling -> Affine-ReLU -> Affine-Softmax

In [ ]:
from collections import OrderedDict

class SimpleConvNet:
  def __init__(self, 
                input_dim=(1, 28, 28), 
                conv_param={
                 'filter_num':30, 
                 'filter_size':5, 
                 'pad': 0, 
                 'stride':1
                },
                hidden_size=100, output_Size=10, weight_init_std=0.01):
    # size 세팅
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
    pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))
    
    # 가중치 세팅
    self.params = {}
    self.params['W1'] = \
      weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = \
      weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(pool_output_size)
    self.params['W3'] = \
      weight_init_std * np.random.randn(hidden_size, output_Size)
    self.params['b3'] = np.zeros(output_Size)
    
    # layer 세팅
    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], 
                                       conv_param['stride'], conv_param['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, strid=2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    
    self.last_layer = SoftmaxWithLoss()
    
  def predict(self, x):
    for layer in self.layers:
      x = layer.forward(x)
      
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)
  
  def gradient(self, x, t):
    self.loss(x, t)
    
    dout = 1
    dout = self.last_layer.backward(dout)
    for layer in self.layers:
      dout = 
  